In [ ]:
from virelay import VirelayLazy

In [ ]:
from torchvision.models import resnet18
import torch
from torchvision import transforms, datasets, utils
import os

model = resnet18()
model.fc.out_features = 10 
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
labels = [{"index" : 0, "word_net_id": "1", "name": "airplane"}, #word_net_id does not matter but needs to be filled in
          {"index": 1, "word_net_id": "2", "name":"automobile"},
          {"index" : 2, "word_net_id": "3", "name": "bird"},
          {"index" : 3, "word_net_id": "4", "name": "cat"},
          {"index" : 4, "word_net_id": "5", "name": "deer"},
          {"index" : 5, "word_net_id": "6", "name": "dog"},
          {"index" : 6, "word_net_id": "7", "name": "frog"},
          {"index" : 7, "word_net_id": "8", "name": "horse"},
          {"index" : 8, "word_net_id": "9", "name": "ship"},
          {"index" : 9, "word_net_id": "10", "name": "truck"}
          ]

# Define the transformations
transform = transforms.Compose(
            [
                transforms.Resize((32,32)),
                transforms.ToTensor(),
                transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]),
            ]
        )


In [ ]:
# Define the root directory to save the dataset
root_dir = "demo/cifar10"

# Download the CIFAR10 dataset
testset = datasets.CIFAR10(root=root_dir, train=False, transform=transform, download=True)

# Create a folder for each class
classes = testset.classes
for i, class_name in enumerate(classes):
    class_dir = os.path.join(root_dir, str(i))
    os.makedirs(class_dir, exist_ok=True)

#Batch to images
for idx, (image, label) in enumerate(testset):
    class_name = classes[label]
    image_path = os.path.join(root_dir, str(label), f"test_{idx}.png")
    utils.save_image(image, image_path)

import shutil
shutil.rmtree(os.path.join(root_dir, "cifar-10-batches-py"))


In [ ]:
#model, dataset_path, model_weights_path, device, labels/num_classes, project_folder
vir = VirelayLazy(model, "demo/cifar10", transform, model_weights_path = None, labels=labels, project_folder="demo/")

In [ ]:
from zennit.composites import EpsilonGammaBox, EpsilonAlpha2Beta1, EpsilonPlus

composite = EpsilonGammaBox(-3.0, 3.0)
vir.compute_attributions(32, 1000, composite, attribution_file_name="attribution.h5", attributor_name="gradient")

In [ ]:
vir.set_attribution_path("demo/attribution.h5") # If you have computed attributions already
#Each project can have multiple meta analyses
vir.add_meta_analysis(variant="spectral", analysis_file_name="spectral.h5", number_of_eigenvalues=32, number_of_clusters_list=list(range(2,31)), number_of_neighbors=32)
#vir.add_meta_analysis(variant="fullspectral", analysis_file_name="fullspectral.h5", number_of_eigenvalues=32)

In [ ]:

# The Virelay app is based of off a project file
vir.make_project_file("^demo/cifar10\/(\d+)\/.+$", dataset_down_sampling_method="resize")

In [ ]:
#Start the web app
vir.start_virelay(project_file="demo/project.yaml", host="localhost", port=8080, debug_mode=False)